In [ ]:
#REvisas como Mejorar Pruebas....
import os
import json
import time
import pandas as pd
from openai import OpenAI
from tqdm import tqdm
from openpyxl import load_workbook

# === CARGA CONFIGURACIÓN ===
CONFIG_PATH = "config.json"

if not os.path.exists(CONFIG_PATH):
    raise FileNotFoundError("❌ No se encontró el archivo config.json")

with open(CONFIG_PATH, "r", encoding="utf-8") as f:
    config = json.load(f)

API_KEY = config["openai_api_key"]
MODEL = config["model"]
input_cfg = config["input_settings"]
output_cfg = config["output_settings"]
logging_cfg = config["logging"]

client = OpenAI(api_key=API_KEY)

# === PROMPT DEL MODELO ===
SYSTEM_PROMPT = """

Eres un Especialista en costos de repuestos de maquinaria industrial, Experto en compras internacionales y cálculo de costos de reposición de inventarios.

Tu tarea es calcular un costo de reposición sostenible (no el más barato, sino el más estable (equilibrio sostenible entre precio y disponibilidad) en el tiempo)
y distribuir la demanda entre los tres orígenes (USA, BR, EURO).

---

### Reglas principales

1. Los datos de costos y disponibilidades son 100% confiables.  
   NO hay sesgo geográfico (NINGÚN país es más estable por razones económicas, políticas o históricas).  
   Tu meta no es optimizar el precio, sino encontrar un costo de reposición estable basado en el equilibrio sostenible entre precio y disponibilidad para 6–12 meses.  

   Antes de cualquier cálculo, valida coherencia por origen: cada costo solo puede aplicarse a su propia disponibilidad.  
   Cada origen tiene su propio costo y su propia disponibilidad, y ambos son inseparables.  
   Si un origen no tiene stock, ignora su costo; nunca lo uses en otro.  
   Jamás asignes abastecimiento de un origen usando el costo de otro.  
   El costo USA pertenece exclusivamente a USA, el costo BR a BR y el costo EURO a EURO.  
   Si alguna decisión requiere mezclar, promediar o sustituir costos entre orígenes, **no la ejecutes** y devuelve alerta "REV_HUMANO" con categoría "Crítico".  
   El costo de reposición debe calcularse únicamente con los orígenes válidos (costo y disponibilidad del mismo origen).


   Si se detecta una situación que requiera usar costos cruzados entre orígenes, **no lo hagas**: clasifica el caso con la alerta `"REV_HUMANO"` y la categoría `"Crítico"`.

2. Si un origen tiene disponibilidad suficiente y precio coherente (preferiblemente el mas bajo), puede ser el principal.  
   No generes alerta aunque haya otros más caros.

3. Si el origen más barato no cubre toda la demanda, reparte el faltante entre los demás orígenes con disponibilidad real.

4. No dependas 100 % de un solo origen: deja margen de reserva.

5. Si la disponibilidad total es baja o los precios son muy dispares, devuelve una alerta  
   “precio no sostenible” o “revisar con agente humano”.

6. Si los precios son parecidos y la disponibilidad combinada cubre la demanda,  
   distribuye proporcionalmente y calcula un promedio ponderado.

---

### Proyección a futuro
Evalúa la probabilidad de que el costo cambie en los próximos 6 meses (Baja, Media o Alta)  
y calcula dos escenarios:

- `C_Repo_Pesimista`: si desaparece el origen más barato.  
- `C_Repo_Optimista`: si aumenta la disponibilidad del origen más barato.

---

### Formato obligatorio de salida
Devuelve SIEMPRE un JSON:
{
  "Abast_USA": <int>,
  "Abast_BR": <int>,
  "Abast_EURO": <int>,
  "C_Repo_Prob": <float>,
  "C_Repo_Pesimista": <float>,
  "C_Repo_Optimista": <float>,
  "probabilidad_cambio_costo": "<string>",

  "alertas": [
    // === DISPONIBILIDAD ===
    // DISP_ORIGEN_BAJA → disponibilidad baja en uno o más orígenes.
    // DISP_BAJA_GENERAL → pocas unidades en todos los orígenes.
    // DISP_UNICO_ORIGEN → solo un origen tiene disponibilidad (riesgo de dependencia).
    // DISP_DESBAL → desequilibrio severo entre orígenes (uno con stock alto, otros vacíos).
    // DISP_SOBREASIGNADA → exceso de abastecimiento desde un solo origen (riesgo operativo).

    // === PRECIO Y MERCADO ===
    // PRECIO_ALTO → precios elevados pero con disponibilidad suficiente (mercado caro pero estable).
    // PRECIO_ALTO_NODISP → precios altos y pocas unidades disponibles (mercado restringido).
    // PRECIO_DISPAR → precios incoherentes o sin correlación con disponibilidad.
    // PRECIO_BAJO → precio seleccionado muy bajo, pero con riesgo por poca disponibilidad.
    // MERCADO_TENSIONADO → todos los orígenes muestran baja disponibilidad y precios altos; riesgo sistémico.

    // === COBERTURA Y DECISIÓN ===
    // SIN_COBERTURA → ningún origen puede cubrir la demanda.
    // REV_HUMANO → requiere revisión manual (situación anómala).
    // NINGUNA → sin alertas; mercado estable.
],

  "categoria": "<string>" // ejemplo: "Sostenible", "Inestable", "Crítico", "Revisar"
}
"""

# === FUNCIÓN PARA LEER COLUMNA POR LETRA ===
def col_letter_to_index(letter: str) -> int:
    """Convierte letras tipo Excel (A, B, AA) en índice base 0."""
    letter = letter.upper()
    result = 0
    for ch in letter:
        result = result * 26 + (ord(ch) - ord("A")) + 1
    return result - 1

# === LECTURA DEL EXCEL ===
input_file = input_cfg["input_excel_file"]
header_row = input_cfg["header_row"]
col_map = input_cfg["columns"]

print(f"📂 Leyendo archivo: {input_file}")

df = pd.read_excel(input_file, header=header_row).head(20)

# Asignar nombres fijos a las columnas según el mapeo
df = df.rename(columns={
    df.columns[col_letter_to_index(col_map["COD"])]: "COD",
    df.columns[col_letter_to_index(col_map["Cantidad_Solicitada"])]: "Demanda",
    df.columns[col_letter_to_index(col_map["Costo_USA"])]: "Costo_USA",
    df.columns[col_letter_to_index(col_map["Costo_BR"])]: "Costo_BR",
    df.columns[col_letter_to_index(col_map["Costo_EURO"])]: "Costo_EURO",
    df.columns[col_letter_to_index(col_map["DispEff_USA"])]: "DispEff_USA",
    df.columns[col_letter_to_index(col_map["DispEff_BR"])]: "DispEff_BR",
    df.columns[col_letter_to_index(col_map["DispEff_EURO"])]: "DispEff_EURO"
})

# === FUNCIÓN PARA PROCESAR UN SKU ===
def procesar_item(row):
    user_prompt = {
        "SKU": str(row["COD"]),
        "Demanda": float(row["Demanda"]),
        "Costos": {
            "USA": float(row["Costo_USA"]),
            "BR": float(row["Costo_BR"]),
            "EURO": float(row["Costo_EURO"])
        },
        "Disponibilidades": {
            "USA": float(row["DispEff_USA"]),
            "BR": float(row["DispEff_BR"]),
            "EURO": float(row["DispEff_EURO"])
        }
    }

    try:
        response = client.chat.completions.create(
            model=MODEL,
            temperature=0,
            top_p=1,
            seed=1234,            
            response_format={"type": "json_object"},
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT.strip()},
                {"role": "user", "content": json.dumps(user_prompt, ensure_ascii=False)}
            ]
        )

        result = json.loads(response.choices[0].message.content)

        return {
            "COD": user_prompt["SKU"],
            "Abast_USA": result.get("Abast_USA", 0),
            "Abast_BR": result.get("Abast_BR", 0),
            "Abast_EURO": result.get("Abast_EURO", 0),
            "C_Repo_Prob": result.get("C_Repo_Prob", 0.0),
            "C_Repo_Pesimista": result.get("C_Repo_Pesimista", 0.0),
            "C_Repo_Optimista": result.get("C_Repo_Optimista", 0.0),
            "probabilidad_cambio_costo": result.get("probabilidad_cambio_costo", "Desconocida"),
            "alertas": ", ".join(result.get("alertas", [])),
            "categoria": result.get("categoria", "")
        }

    except Exception as e:
        if logging_cfg["enable_logs"]:
            with open(logging_cfg["log_file"], "a", encoding="utf-8") as logf:
                logf.write(f"Error con SKU {user_prompt['SKU']}: {str(e)}\n")

        return {
            "COD": user_prompt["SKU"],
            "error": str(e)
        }

# === PROCESAR TODOS LOS SKUS ===
print("⚙️ Procesando SKUs...")

resultados = []
for _, row in tqdm(df.iterrows(), total=len(df)):
    resultados.append(procesar_item(row))
    time.sleep(0.1)

df_result = pd.DataFrame(resultados)

# === GUARDAR RESULTADOS ===
output_file = output_cfg["excel_output_file"]
print(f"💾 Guardando resultados en: {output_file}")
df_result.to_excel(output_file, index=False)

# === OPCIONAL: INSERTAR EN EL ARCHIVO ORIGINAL ===
try:
    wb = load_workbook(input_file)
    ws = wb.active

    start_col = col_letter_to_index(output_cfg["columns_output_start"]) + 1
    start_row = header_row + 1  # Fila donde empiezan los datos

    headers = list(df_result.columns[1:])  # sin COD
    for i, header in enumerate(headers):
        ws.cell(row=header_row + 1, column=start_col + i, value=header)

    for r_idx, row in enumerate(df_result.itertuples(index=False), start=start_row):
        for c_idx, value in enumerate(row[1:], start=start_col):
            ws.cell(row=r_idx, column=c_idx, value=value)
 
    wb.save(input_file)
    print("✅ Resultados insertados en el archivo original con éxito.")
except Exception as e:
    print(f"⚠️ No se pudo insertar en el archivo original: {e}")

print("🏁 Proceso finalizado.")


📂 Leyendo archivo: Modelo_Precios_Peru.xlsx
⚙️ Procesando SKUs...


100%|██████████| 20/20 [00:58<00:00,  2.92s/it]


💾 Guardando resultados en: resultado_reposicion_v3.xlsx
✅ Resultados insertados en el archivo original con éxito.
🏁 Proceso finalizado.


In [ ]:
##FUNCIONA PARCIALMENTE ASIGNA un % al mejor pero no lo cosume completo

import os
import json
import time
import pandas as pd
from openai import OpenAI
from tqdm import tqdm
from openpyxl import load_workbook

# === CARGA CONFIGURACIÓN ===
CONFIG_PATH = "config.json"

if not os.path.exists(CONFIG_PATH):
    raise FileNotFoundError("❌ No se encontró el archivo config.json")

with open(CONFIG_PATH, "r", encoding="utf-8") as f:
    config = json.load(f)

API_KEY = config["openai_api_key"]
MODEL = config["model"]
input_cfg = config["input_settings"]
output_cfg = config["output_settings"]
logging_cfg = config["logging"]

client = OpenAI(api_key=API_KEY)


# === PROMPT DEL MODELO ===
SYSTEM_PROMPT = """
Eres un Especialista experto en costos industriales y reposición de inventarios.

Tu tarea es calcular un costo de reposición sostenible (no el más barato, sino el más estable, logrando un equilibrio sostenible entre precio y disponibilidad en un plazo de 6-12 meses)
y distribuir la demanda entre los tres orígenes (USA, BR, EURO).

---

### Reglas principales

1. Los datos de costos y disponibilidades son 100% confiables.  
   NO hay sesgo geográfico (NINGÚN país es más estable por razones económicas, políticas o históricas).  
   Tu meta no es optimizar el precio, sino encontrar un costo de reposición estable basado en el equilibrio sostenible entre precio y disponibilidad para 6–12 meses.  
   Cada origen tiene su propio costo y su propia disponibilidad.  
   Jamás asignes abastecimiento de un origen usando el costo de otro.  
   El costo USA pertenece exclusivamente a USA, el costo BR a BR y el costo EURO a EURO.

2. Si un origen tiene disponibilidad suficiente y precio coherente (aunque sea el más alto), puede ser el principal y se considera sostenible.  
   No generes alerta aunque haya otros más baratos.

3. Si el origen más barato apenas cubre la demanda y parece outlier, limita su asignación y reparte el resto entre orígenes con stock real.

3.1 Nunca combines precios y disponibilidades entre orígenes:  
Si asignas unidades a un origen, el costo asociado debe ser EXCLUSIVAMENTE el de ese mismo origen.  
Nunca mezcles el costo de un origen con unidades de otro.  
Si hay disponibilidad en varios orígenes, cada uno debe contribuir solo con su propio costo y su propia cantidad.

4. Si la demanda esta justa con la disponibilidad No dependas 100 % de un solo origen: deja margen de reserva.

5. Marca alerta solo si la disponibilidad total es baja o los precios son extremadamente dispares y las disponibilidades son limitadas.  
   En esos casos devuelve “precio no sostenible” o “revisar con agente humano”.

6. Si los precios son parecidos y la disponibilidad combinada cubre la demanda,  
   distribuye proporcionalmente y calcula un promedio ponderado.

---

### Proyección a futuro
Evalúa la probabilidad de que el costo cambie en los próximos 6 meses (Baja, Media o Alta)  
y calcula dos escenarios:

- `C_Repo_Pesimista`: si desaparece el origen más barato.  
- `C_Repo_Optimista`: si aumenta la disponibilidad del origen más barato.

---

### Formato obligatorio de salida
Devuelve SIEMPRE un JSON:
{
  "Abast_USA": <int>,
  "Abast_BR": <int>,
  "Abast_EURO": <int>,
  "C_Repo_Prob": <float>,
  "C_Repo_Pesimista": <float>,
  "C_Repo_Optimista": <float>,
  "probabilidad_cambio_costo": "<string>",

  "alertas": [
    // === DISPONIBILIDAD ===
    // DISP_ORIGEN_BAJA → disponibilidad baja en uno o más orígenes.
    // DISP_BAJA_GENERAL → pocas unidades en todos los orígenes.
    // DISP_UNICO_ORIGEN → solo un origen tiene disponibilidad (riesgo de dependencia).
    // DISP_DESBAL → desequilibrio severo entre orígenes (uno con stock alto, otros vacíos).
    // DISP_SOBREASIGNADA → exceso de abastecimiento desde un solo origen (riesgo operativo).

    // === PRECIO Y MERCADO ===
    // PRECIO_ALTO → precios elevados pero con disponibilidad suficiente (mercado caro pero estable).
    // PRECIO_ALTO_NODISP → precios altos y pocas unidades disponibles (mercado restringido).
    // PRECIO_DISPAR → precios incoherentes o sin correlación con disponibilidad.
    // PRECIO_BAJO → precio seleccionado muy bajo, pero con riesgo por poca disponibilidad.
    // MERCADO_TENSIONADO → todos los orígenes muestran baja disponibilidad y precios altos; riesgo sistémico.

    // === COBERTURA Y DECISIÓN ===
    // SIN_COBERTURA → ningún origen puede cubrir la demanda.
    // REV_HUMANO → requiere revisión manual (situación anómala).
    // NINGUNA → sin alertas; mercado estable.
],

  "categoria": "<string>" // ejemplo: "Sostenible", "Inestable", "Crítico", "Revisar"
}
"""


# === FUNCIÓN PARA LEER COLUMNA POR LETRA ===
def col_letter_to_index(letter: str) -> int:
    """Convierte letras tipo Excel (A, B, AA) en índice base 0."""
    letter = letter.upper()
    result = 0
    for ch in letter:
        result = result * 26 + (ord(ch) - ord("A")) + 1
    return result - 1

# === LECTURA DEL EXCEL ===
input_file = input_cfg["input_excel_file"]
header_row = input_cfg["header_row"]
col_map = input_cfg["columns"]

print(f"📂 Leyendo archivo: {input_file}")

df = pd.read_excel(input_file, header=header_row).head(20)

# Asignar nombres fijos a las columnas según el mapeo
df = df.rename(columns={
    df.columns[col_letter_to_index(col_map["COD"])]: "COD",
    df.columns[col_letter_to_index(col_map["Cantidad_Solicitada"])]: "Demanda",
    df.columns[col_letter_to_index(col_map["Costo_USA"])]: "Costo_USA",
    df.columns[col_letter_to_index(col_map["Costo_BR"])]: "Costo_BR",
    df.columns[col_letter_to_index(col_map["Costo_EURO"])]: "Costo_EURO",
    df.columns[col_letter_to_index(col_map["DispEff_USA"])]: "DispEff_USA",
    df.columns[col_letter_to_index(col_map["DispEff_BR"])]: "DispEff_BR",
    df.columns[col_letter_to_index(col_map["DispEff_EURO"])]: "DispEff_EURO"
})

# === FUNCIÓN PARA PROCESAR UN SKU ===
def procesar_item(row):
    user_prompt = {
        "SKU": str(row["COD"]),
        "Demanda": float(row["Demanda"]),
        "Costos": {
            "USA": float(row["Costo_USA"]),
            "BR": float(row["Costo_BR"]),
            "EURO": float(row["Costo_EURO"])
        },
        "Disponibilidades": {
            "USA": float(row["DispEff_USA"]),
            "BR": float(row["DispEff_BR"]),
            "EURO": float(row["DispEff_EURO"])
        }
    }

    try:
        response = client.chat.completions.create(
            model=MODEL,
            temperature=0,
            top_p=1,
            seed=1234,            
            response_format={"type": "json_object"},
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT.strip()},
                {"role": "user", "content": json.dumps(user_prompt, ensure_ascii=False)}
            ]
        )

        result = json.loads(response.choices[0].message.content)

        return {
            "COD": user_prompt["SKU"],
            "Abast_USA": result.get("Abast_USA", 0),
            "Abast_BR": result.get("Abast_BR", 0),
            "Abast_EURO": result.get("Abast_EURO", 0),
            "C_Repo_Prob": result.get("C_Repo_Prob", 0.0),
            "C_Repo_Pesimista": result.get("C_Repo_Pesimista", 0.0),
            "C_Repo_Optimista": result.get("C_Repo_Optimista", 0.0),
            "probabilidad_cambio_costo": result.get("probabilidad_cambio_costo", "Desconocida"),
            "alertas": ", ".join(result.get("alertas", [])),
            "categoria": result.get("categoria", "")
        }

    except Exception as e:
        if logging_cfg["enable_logs"]:
            with open(logging_cfg["log_file"], "a", encoding="utf-8") as logf:
                logf.write(f"Error con SKU {user_prompt['SKU']}: {str(e)}\n")

        return {
            "COD": user_prompt["SKU"],
            "error": str(e)
        }

# === PROCESAR TODOS LOS SKUS ===
print("⚙️ Procesando SKUs...")

resultados = []
for _, row in tqdm(df.iterrows(), total=len(df)):
    resultados.append(procesar_item(row))
    time.sleep(0.1)

df_result = pd.DataFrame(resultados)

# === GUARDAR RESULTADOS ===
output_file = output_cfg["excel_output_file"]
print(f"💾 Guardando resultados en: {output_file}")
df_result.to_excel(output_file, index=False)

# === OPCIONAL: INSERTAR EN EL ARCHIVO ORIGINAL ===
try:
    wb = load_workbook(input_file)
    ws = wb.active

    start_col = col_letter_to_index(output_cfg["columns_output_start"]) + 1
    start_row = header_row + 1  # Fila donde empiezan los datos

    headers = list(df_result.columns[1:])  # sin COD
    for i, header in enumerate(headers):
        ws.cell(row=header_row + 1, column=start_col + i, value=header)

    for r_idx, row in enumerate(df_result.itertuples(index=False), start=start_row):
        for c_idx, value in enumerate(row[1:], start=start_col):
            ws.cell(row=r_idx, column=c_idx, value=value)
 
    wb.save(input_file)
    print("✅ Resultados insertados en el archivo original con éxito.")
except Exception as e:
    print(f"⚠️ No se pudo insertar en el archivo original: {e}")

print("🏁 Proceso finalizado.")

📂 Leyendo archivo: Modelo_Precios_Peru.xlsx
⚙️ Procesando SKUs...


100%|██████████| 20/20 [00:54<00:00,  2.72s/it]


💾 Guardando resultados en: resultado_reposicion_v3.xlsx
✅ Resultados insertados en el archivo original con éxito.
🏁 Proceso finalizado.


In [48]:
###FUNCIONA COMO QUIERO PARA EL 84214564

import os
import json
import time
import pandas as pd
from openai import OpenAI
from tqdm import tqdm
from openpyxl import load_workbook

# === CARGA CONFIGURACIÓN ===
CONFIG_PATH = "config.json"

if not os.path.exists(CONFIG_PATH):
    raise FileNotFoundError("❌ No se encontró el archivo config.json")

with open(CONFIG_PATH, "r", encoding="utf-8") as f:
    config = json.load(f)

API_KEY = config["openai_api_key"]
MODEL = config["model"]
input_cfg = config["input_settings"]
output_cfg = config["output_settings"]
logging_cfg = config["logging"]

client = OpenAI(api_key=API_KEY)

SYSTEM_PROMPT = """

Eres un Especialista en costos de repuestos de maquinaria industrial, experto en compras internacionales y cálculo de costos de reposición de inventarios.

Tu tarea es calcular un costo de reposición sostenible —no el más barato, sino el más estable en el tiempo— logrando equilibrio entre precio y disponibilidad en un horizonte de 6–12 meses.  
También debes distribuir la demanda entre los tres orígenes: USA, BR y EURO.

---

### Reglas principales

1. Los datos de costos y disponibilidades son confiables y deben tratarse como valores independientes por origen.  
   No existe sesgo geográfico: ningún país es más estable por razones económicas, políticas o históricas.  
   El objetivo no es obtener el menor precio, sino calcular un costo de reposición estable y sostenible (equilibrio entre precio y disponibilidad) en un horizonte de 6–12 meses.  
   Cada origen tiene su propio costo y disponibilidad, y ambos son indivisibles.  

   Bajo ninguna circunstancia está permitido asignar abastecimiento de un origen utilizando el costo de otro.  
   Mezclar, promediar o sustituir precios entre orígenes constituye una violación crítica de las reglas.  
   - El costo USA pertenece exclusivamente a USA.  
   - El costo BR pertenece exclusivamente a BR.  
   - El costo EURO pertenece exclusivamente a EURO.  

   Si se detecta una situación que implique usar costos cruzados entre orígenes, **no lo hagas**: clasifica el caso con la alerta `"REV_HUMANO"` y la categoría `"Crítico"`.

2. Si un origen tiene disponibilidad suficiente y precio coherente (aunque no sea el más bajo), puede ser el principal.  
   No generes alerta aunque existan otros más caros.

3. Si el origen más barato no cubre toda la demanda, reparte el faltante entre los demás orígenes con disponibilidad real.

4. Evita depender totalmente de un solo origen cuando la disponibilidad total sea ajustada; deja margen de reserva.

5. Si la disponibilidad total es baja o los precios son muy dispares, devuelve una alerta `"precio no sostenible"` o `"revisar con agente humano"`.

6. Si los precios son similares y la disponibilidad combinada cubre la demanda, distribuye proporcionalmente y calcula un promedio ponderado.

---

### Proyección a futuro
Evalúa la probabilidad de que el costo cambie en los próximos meses (Baja, Media o Alta)  
y calcula dos escenarios:

- `C_Repo_Pesimista`: si desaparece el origen más barato.  
- `C_Repo_Optimista`: si aumenta la disponibilidad del origen más competitivo.

---

### Formato obligatorio de salida
Devuelve SIEMPRE un JSON:
{
  "Abast_USA": <int>,
  "Abast_BR": <int>,
  "Abast_EURO": <int>,
  "C_Repo_Prob": <float>,
  "C_Repo_Pesimista": <float>,
  "C_Repo_Optimista": <float>,
  "probabilidad_cambio_costo": "<string>",

  "alertas": [
    // === DISPONIBILIDAD ===
    // DISP_ORIGEN_BAJA → disponibilidad baja en uno o más orígenes.
    // DISP_BAJA_GENERAL → pocas unidades en todos los orígenes.
    // DISP_UNICO_ORIGEN → solo un origen con disponibilidad (riesgo de dependencia).
    // DISP_DESBAL → desequilibrio severo entre orígenes.
    // DISP_SOBREASIGNADA → exceso de abastecimiento desde un solo origen.

    // === PRECIO Y MERCADO ===
    // PRECIO_ALTO → precios elevados pero con disponibilidad suficiente.
    // PRECIO_ALTO_NODISP → precios altos y baja disponibilidad.
    // PRECIO_DISPAR → precios incoherentes o sin correlación con disponibilidad.
    // PRECIO_BAJO → precio muy bajo con riesgo por poca disponibilidad.
    // MERCADO_TENSIONADO → baja disponibilidad general y precios altos; riesgo sistémico.

    // === COBERTURA Y DECISIÓN ===
    // SIN_COBERTURA → ningún origen cubre la demanda.
    // REV_HUMANO → requiere revisión manual (situación anómala).
    // NINGUNA → sin alertas; mercado estable.
  ],

  "categoria": "<string>" // ejemplo: "Sostenible", "Inestable", "Crítico", "Revisar"
}
"""




# === FUNCIÓN PARA LEER COLUMNA POR LETRA ===
def col_letter_to_index(letter: str) -> int:
    """Convierte letras tipo Excel (A, B, AA) en índice base 0."""
    letter = letter.upper()
    result = 0
    for ch in letter:
        result = result * 26 + (ord(ch) - ord("A")) + 1
    return result - 1

# === LECTURA DEL EXCEL ===
input_file = input_cfg["input_excel_file"]
header_row = input_cfg["header_row"]
col_map = input_cfg["columns"]

print(f"📂 Leyendo archivo: {input_file}")

df = pd.read_excel(input_file, header=header_row).head(20)

# Asignar nombres fijos a las columnas según el mapeo
df = df.rename(columns={
    df.columns[col_letter_to_index(col_map["COD"])]: "COD",
    df.columns[col_letter_to_index(col_map["Cantidad_Solicitada"])]: "Demanda",
    df.columns[col_letter_to_index(col_map["Costo_USA"])]: "Costo_USA",
    df.columns[col_letter_to_index(col_map["Costo_BR"])]: "Costo_BR",
    df.columns[col_letter_to_index(col_map["Costo_EURO"])]: "Costo_EURO",
    df.columns[col_letter_to_index(col_map["DispEff_USA"])]: "DispEff_USA",
    df.columns[col_letter_to_index(col_map["DispEff_BR"])]: "DispEff_BR",
    df.columns[col_letter_to_index(col_map["DispEff_EURO"])]: "DispEff_EURO"
})

# === FUNCIÓN PARA PROCESAR UN SKU ===
def procesar_item(row):
    user_prompt = {
        "SKU": str(row["COD"]),
        "Demanda": float(row["Demanda"]),
        "Costos": {
            "USA": float(row["Costo_USA"]),
            "BR": float(row["Costo_BR"]),
            "EURO": float(row["Costo_EURO"])
        },
        "Disponibilidades": {
            "USA": float(row["DispEff_USA"]),
            "BR": float(row["DispEff_BR"]),
            "EURO": float(row["DispEff_EURO"])
        }
    }

    try:
        response = client.chat.completions.create(
            model=MODEL,
            temperature=0,
            top_p=1,
            seed=1234,            
            response_format={"type": "json_object"},
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT.strip()},
                {"role": "user", "content": json.dumps(user_prompt, ensure_ascii=False)}
            ]
        )

        result = json.loads(response.choices[0].message.content)

        return {
            "COD": user_prompt["SKU"],
            "Abast_USA": result.get("Abast_USA", 0),
            "Abast_BR": result.get("Abast_BR", 0),
            "Abast_EURO": result.get("Abast_EURO", 0),
            "C_Repo_Prob": result.get("C_Repo_Prob", 0.0),
            "C_Repo_Pesimista": result.get("C_Repo_Pesimista", 0.0),
            "C_Repo_Optimista": result.get("C_Repo_Optimista", 0.0),
            "probabilidad_cambio_costo": result.get("probabilidad_cambio_costo", "Desconocida"),
            "alertas": ", ".join(result.get("alertas", [])),
            "categoria": result.get("categoria", "")
        }

    except Exception as e:
        if logging_cfg["enable_logs"]:
            with open(logging_cfg["log_file"], "a", encoding="utf-8") as logf:
                logf.write(f"Error con SKU {user_prompt['SKU']}: {str(e)}\n")

        return {
            "COD": user_prompt["SKU"],
            "error": str(e)
        }

# === PROCESAR TODOS LOS SKUS ===
print("⚙️ Procesando SKUs...")

resultados = []
for _, row in tqdm(df.iterrows(), total=len(df)):
    resultados.append(procesar_item(row))
    time.sleep(0.1)

df_result = pd.DataFrame(resultados)

# === GUARDAR RESULTADOS ===
output_file = output_cfg["excel_output_file"]
print(f"💾 Guardando resultados en: {output_file}")
df_result.to_excel(output_file, index=False)

# === OPCIONAL: INSERTAR EN EL ARCHIVO ORIGINAL ===
try:
    wb = load_workbook(input_file)
    ws = wb.active

    start_col = col_letter_to_index(output_cfg["columns_output_start"]) + 1
    start_row = header_row + 1  # Fila donde empiezan los datos

    headers = list(df_result.columns[1:])  # sin COD
    for i, header in enumerate(headers):
        ws.cell(row=header_row + 1, column=start_col + i, value=header)

    for r_idx, row in enumerate(df_result.itertuples(index=False), start=start_row):
        for c_idx, value in enumerate(row[1:], start=start_col):
            ws.cell(row=r_idx, column=c_idx, value=value)
 
    wb.save(input_file)
    print("✅ Resultados insertados en el archivo original con éxito.")
except Exception as e:
    print(f"⚠️ No se pudo insertar en el archivo original: {e}")

print("🏁 Proceso finalizado.")

📂 Leyendo archivo: Modelo_Precios_Peru.xlsx
⚙️ Procesando SKUs...


100%|██████████| 20/20 [00:51<00:00,  2.58s/it]


💾 Guardando resultados en: resultado_reposicion_v3.xlsx
✅ Resultados insertados en el archivo original con éxito.
🏁 Proceso finalizado.
